In [1]:
import pandas as pd
import numpy as np

import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

from boruta import BorutaPy as boruta
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV

Using TensorFlow backend.


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

In [16]:
data = pd.read_csv('../Details/dataset/SentimentbasedRecoEngine/sample30.csv', index_col=None)
data.shape

(30000, 15)

In [17]:
print(data['manufacturer'].nunique())
print(data['name'].nunique())
print(data['categories'].nunique())
print(data['brand'].nunique())

227
271
270
214


In [18]:
df = data[(data['reviews_username'].isnull()==False) & (data['user_sentiment'].isnull()==False)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29936 entries, 0 to 29999
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    29936 non-null  object
 1   brand                 29936 non-null  object
 2   categories            29936 non-null  object
 3   manufacturer          29795 non-null  object
 4   name                  29936 non-null  object
 5   reviews_date          29896 non-null  object
 6   reviews_didPurchase   15931 non-null  object
 7   reviews_doRecommend   27395 non-null  object
 8   reviews_rating        29936 non-null  int64 
 9   reviews_text          29936 non-null  object
 10  reviews_title         29747 non-null  object
 11  reviews_userCity      1900 non-null   object
 12  reviews_userProvince  166 non-null    object
 13  reviews_username      29936 non-null  object
 14  user_sentiment        29936 non-null  object
dtypes: int64(1), object(14)
memory usage

In [19]:
df['user_sentiment'].value_counts()

Positive    26579
Negative     3357
Name: user_sentiment, dtype: int64

In [20]:
df.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


In [23]:
df.groupby(['user_sentiment', 'reviews_doRecommend'])['id'].count()

user_sentiment  reviews_doRecommend
Negative        False                    556
                True                    2380
Positive        False                    994
                True                   23465
Name: id, dtype: int64

In [24]:
df = df[['id', 'brand', 'categories', 'manufacturer', 'name', 'reviews_text', 'reviews_title', 'reviews_doRecommend',
         'reviews_rating', 'user_sentiment']]
df.shape

(29936, 10)

In [25]:
df.isnull().sum()

id                        0
brand                     0
categories                0
manufacturer            141
name                      0
reviews_text              0
reviews_title           189
reviews_doRecommend    2541
reviews_rating            0
user_sentiment            0
dtype: int64

In [26]:
df['reviews_title'] = df['reviews_title'].astype('O')
cnt = [i for i in df['reviews_title'].to_list() if isinstance(i, float)]
print(cnt)
cnt = [i for i in df['reviews_text'].to_list() if i.isnumeric()]
print(cnt)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[]


D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
df['reviews_title'] = df['reviews_title'].fillna('NF')
df = df.reset_index(drop=True)

D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
for i in range(len(df)):
  if df.loc[i,'reviews_title'] == 'NF':
    if df.loc[i, 'user_sentiment'] == 'Positive':
      df.loc[i,'reviews_title']='Good'
    if df.loc[i, 'user_sentiment'] == 'Negative':
      df.loc[i,'reviews_title']='Bad'

df['manufacturer'].fillna(df['brand'], inplace=True)

In [29]:
stemmer = PorterStemmer()

# add stemming and lemmatisation in the preprocess function
def preprocess(document):
    'changes document to lower case and removes stopwords'
    document = document.lower()
    words = word_tokenize(document)
    words = [word for word in words if word not in stopwords.words("english")]
    words = [stemmer.stem(word) for word in words]
    document = " ".join(words)
    return document

In [30]:
df['reviews_text'] = df['reviews_title']+' '+df['reviews_text']
df['reviews_text'][0]

"Just Awesome i love this album. it's very good. more to the hip hop side than her current pop sound.. SO HYPE! i listen to this everyday at the gym! i give it 5star rating all the way. her metaphors are just crazy."

In [31]:
df['proc_reviews_text'] = df['reviews_text'].apply(preprocess)
df.shape

(29936, 11)

In [32]:
df.head(2)

,id,brand,categories,manufacturer,name,reviews_text,reviews_title,reviews_doRecommend,reviews_rating,user_sentiment,proc_reviews_text
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),Just Awesome i love this album. it's very good...,Just Awesome,NaN,5,Positive,awesom love album . 's good . hip hop side cur...
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,Good Good flavor. This review was collected as...,Good,NaN,5,Positive,good good flavor . review collect part promot .


In [33]:
# df.to_csv('final_data.csv', index=None)

In [3]:
df = pd.read_csv('../Details/dataset/SentimentbasedRecoEngine/final_data.csv', index_col=None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29936 entries, 0 to 29935
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   29936 non-null  object
 1   brand                29936 non-null  object
 2   categories           29936 non-null  object
 3   manufacturer         29936 non-null  object
 4   name                 29936 non-null  object
 5   reviews_text         29936 non-null  object
 6   reviews_title        29936 non-null  object
 7   reviews_doRecommend  27395 non-null  object
 8   reviews_rating       29936 non-null  int64 
 9   user_sentiment       29936 non-null  object
 10  proc_reviews_text    29936 non-null  object
dtypes: int64(1), object(10)
memory usage: 2.5+ MB


In [4]:
df['user_sentiment'] = df['user_sentiment'].map({'Positive':1, 'Negative':0})
df['user_sentiment'].value_counts()

1    26579
0     3357
Name: user_sentiment, dtype: int64

In [5]:
df.loc[0, 'reviews_text']

"Just Awesome i love this album. it's very good. more to the hip hop side than her current pop sound.. SO HYPE! i listen to this everyday at the gym! i give it 5star rating all the way. her metaphors are just crazy."

In [6]:
df.loc[0, 'proc_reviews_text']

"awesom love album . 's good . hip hop side current pop sound .. hype ! listen everyday gym ! give 5star rate way . metaphor crazi ."

In [7]:
X, y = df['proc_reviews_text'], df['user_sentiment']
xtrain, xtest, ytrain, ytest = train_test_split(df['proc_reviews_text'], df['user_sentiment'], test_size=0.25)

## Feature Creation
1. TF-IDF Vectorizer 
2. Bag-of-Words 

In [37]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_model = tfidf_vectorizer.fit(X)

In [38]:
xtrain_tfi = tfidf_model.transform(xtrain)
xtest_tfi  = tfidf_model.transform(xtest)

xdf_tf = pd.DataFrame(xtrain_tfi.toarray(), columns=tfidf_vectorizer.get_feature_names())
print(xdf_tf.shape)

(22452, 13290)


In [39]:
bow_vectorizer = CountVectorizer(stop_words='english')
bow_model = bow_vectorizer.fit(X)

In [40]:
xtrain_bow = bow_model.transform(xtrain)
xtest_bow  = bow_model.transform(xtest)

xdf_bow = pd.DataFrame(xtrain_bow.toarray(), columns=bow_vectorizer.get_feature_names())
print(xdf_bow.shape)

(22452, 13076)


In [41]:
xtrain_tfi.shape, xtest_tfi.shape, xtrain_bow.shape, xtest_bow.shape

((22452, 13290), (7484, 13290), (22452, 13076), (7484, 13076))

## Class Imbalance Fix
1. TF-IDF features
2. Bag-of-Words features

In [42]:
oversample = SMOTE()
x_train_tf, y_train_tf = oversample.fit_resample(xtrain_tfi, ytrain)

In [43]:
y_train_tf.value_counts(), x_train_tf.shape

(0    19931
 1    19931
 Name: user_sentiment, dtype: int64,
 (39862, 13290))

In [44]:
oversample = SMOTE()
x_train_bw, y_train_bw = oversample.fit_resample(xtrain_bow, ytrain)

In [45]:
y_train_bw.value_counts(), x_train_bw.shape

(0    19931
 1    19931
 Name: user_sentiment, dtype: int64,
 (39862, 13076))

## Modelling
1. TF-IDF
2. Bag-of-Words

### Logistic Regression Model

In [49]:
logreg1 = LogisticRegression(penalty="l2", random_state=42, C=3.5)
logreg1.fit(x_train_tf, y_train_tf)

pred_train = logreg1.predict(x_train_tf)
pred_test  = logreg1.predict(xtest_tfi)

D:\Programs\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [50]:
print("Train Report: {}".format(classification_report(pred_train, y_train_tf)))
print("Test accuracy: {}".format(classification_report(pred_test, ytest)))

Train Report:               precision    recall  f1-score   support

           0       0.97      0.92      0.94     20838
           1       0.92      0.96      0.94     19024

    accuracy                           0.94     39862
   macro avg       0.94      0.94      0.94     39862
weighted avg       0.94      0.94      0.94     39862

Test accuracy:               precision    recall  f1-score   support

           0       0.75      0.45      0.56      1396
           1       0.88      0.97      0.92      6088

    accuracy                           0.87      7484
   macro avg       0.82      0.71      0.74      7484
weighted avg       0.86      0.87      0.86      7484



In [51]:
print(accuracy_score(pred_test, ytest), roc_auc_score(pred_test, ytest))

0.8701229289150187 0.7087261708881014


Old: Train accuracy: 0.8899546647578144 . Test accuracy: 0.8838659392049883
Train AUC: 0.9449458052810008. Test AUC: 0.5255060352831941

In [52]:
logreg2 = LogisticRegression(penalty="l2", random_state=42, C=3.5)
logreg2.fit(x_train_bw, y_train_bw)

pred_train2 = logreg2.predict(x_train_bw)
pred_test2  = logreg2.predict(xtest_bow)

D:\Programs\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [53]:
print("Train Report: {}".format(classification_report(pred_train2, y_train_bw)))
print("Test accuracy: {}".format(classification_report(pred_test2, ytest)))

Train Report:               precision    recall  f1-score   support

           0       0.97      0.96      0.97     20036
           1       0.96      0.97      0.97     19826

    accuracy                           0.97     39862
   macro avg       0.97      0.97      0.97     39862
weighted avg       0.97      0.97      0.97     39862

Test accuracy:               precision    recall  f1-score   support

           0       0.66      0.51      0.58      1076
           1       0.92      0.96      0.94      6408

    accuracy                           0.89      7484
   macro avg       0.79      0.73      0.76      7484
weighted avg       0.88      0.89      0.89      7484



In [54]:
print(accuracy_score(pred_test2, ytest), roc_auc_score(pred_test2, ytest))

0.892036344200962 0.7343457759584905


### Naive Bayes Model

In [85]:
bnb1 = BernoulliNB()
bnb1.fit(x_train_tf, y_train_tf)

pred_train_nb1 = bnb1.predict(x_train_tf)
pred_test_nb1  = bnb1.predict(xtest_tfi)

In [86]:
print("Train Report: {}".format(classification_report(pred_train_nb1, y_train_tf)))
print("Test accuracy: {}".format(classification_report(pred_test_nb1, ytest)))

Train Report:               precision    recall  f1-score   support

           0       0.67      0.88      0.76     15055
           1       0.91      0.73      0.81     24807

    accuracy                           0.79     39862
   macro avg       0.79      0.81      0.79     39862
weighted avg       0.82      0.79      0.79     39862

Test accuracy:               precision    recall  f1-score   support

           0       0.32      0.29      0.30       935
           1       0.90      0.91      0.91      6549

    accuracy                           0.84      7484
   macro avg       0.61      0.60      0.61      7484
weighted avg       0.83      0.84      0.83      7484



In [87]:
print(accuracy_score(pred_test_nb1, ytest), roc_auc_score(pred_test_nb1, ytest))

0.835515766969535 0.6011723225083145


In [88]:
bnb2 = BernoulliNB()
bnb2.fit(x_train_bw, y_train_bw)

pred_train_nb2 = bnb2.predict(x_train_bw)
pred_test_nb2  = bnb2.predict(xtest_bow)

In [89]:
print("Train Report: {}".format(classification_report(pred_train_nb2, y_train_bw)))
print("Test accuracy: {}".format(classification_report(pred_test_nb2, ytest)))

Train Report:               precision    recall  f1-score   support

           0       0.92      0.81      0.86     22807
           1       0.78      0.91      0.84     17055

    accuracy                           0.85     39862
   macro avg       0.85      0.86      0.85     39862
weighted avg       0.86      0.85      0.85     39862

Test accuracy:               precision    recall  f1-score   support

           0       0.49      0.20      0.29      2029
           1       0.76      0.92      0.83      5455

    accuracy                           0.73      7484
   macro avg       0.62      0.56      0.56      7484
weighted avg       0.69      0.73      0.68      7484



In [90]:
print(accuracy_score(pred_test_nb2, ytest), roc_auc_score(pred_test_nb2, ytest))

0.7272848743987173 0.5626644181820071


### XGBoost Classifier

In [91]:
xgb1 = XGBClassifier()
xgb1.fit(x_train_tf, y_train_tf)

pred_train_xg1 = xgb1.predict(x_train_tf)
pred_test_xg1  = xgb1.predict(xtest_tfi)

D:\Programs\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:46:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [92]:
print("Train Report: {}".format(classification_report(pred_train_xg1, y_train_tf)))
print("Test accuracy: {}".format(classification_report(pred_test_xg1, ytest)))

Train Report:               precision    recall  f1-score   support

           0       0.95      0.95      0.95     19817
           1       0.95      0.95      0.95     20045

    accuracy                           0.95     39862
   macro avg       0.95      0.95      0.95     39862
weighted avg       0.95      0.95      0.95     39862

Test accuracy:               precision    recall  f1-score   support

           0       0.61      0.50      0.55      1018
           1       0.92      0.95      0.94      6466

    accuracy                           0.89      7484
   macro avg       0.76      0.72      0.74      7484
weighted avg       0.88      0.89      0.88      7484



In [93]:
print(accuracy_score(pred_test_xg1, ytest), roc_auc_score(pred_test_xg1, ytest))

0.8874933190807055 0.7230084279443874


In [94]:
xgb2 = XGBClassifier()
xgb2.fit(x_train_bw, y_train_bw)

pred_train_xg2 = xgb2.predict(x_train_bw)
pred_test_xg2  = xgb2.predict(xtest_bow)

D:\Programs\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:46:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [95]:
print("Train Report: {}".format(classification_report(pred_train_xg2, y_train_bw)))
print("Test accuracy: {}".format(classification_report(pred_test_xg2, ytest)))

Train Report:               precision    recall  f1-score   support

           0       0.93      0.98      0.95     19063
           1       0.98      0.94      0.96     20799

    accuracy                           0.96     39862
   macro avg       0.96      0.96      0.96     39862
weighted avg       0.96      0.96      0.96     39862

Test accuracy:               precision    recall  f1-score   support

           0       0.45      0.66      0.54       575
           1       0.97      0.93      0.95      6909

    accuracy                           0.91      7484
   macro avg       0.71      0.80      0.74      7484
weighted avg       0.93      0.91      0.92      7484



In [96]:
print(accuracy_score(pred_test_xg2, ytest), roc_auc_score(pred_test_xg2, ytest))

0.9127471940138963 0.7964924138017835


### Random Forest Model

In [97]:
rf1 = RandomForestClassifier()
rf1.fit(x_train_tf, y_train_tf)

pred_train_rf1 = rf1.predict(x_train_tf)
pred_test_rf1  = rf1.predict(xtest_tfi)

In [98]:
print("Train Report: {}".format(classification_report(pred_train_rf1, y_train_tf)))
print("Test accuracy: {}".format(classification_report(pred_test_rf1, ytest)))

Train Report:               precision    recall  f1-score   support

           0       1.00      1.00      1.00     19933
           1       1.00      1.00      1.00     19929

    accuracy                           1.00     39862
   macro avg       1.00      1.00      1.00     39862
weighted avg       1.00      1.00      1.00     39862

Test accuracy:               precision    recall  f1-score   support

           0       0.39      0.56      0.46       576
           1       0.96      0.93      0.94      6908

    accuracy                           0.90      7484
   macro avg       0.68      0.75      0.70      7484
weighted avg       0.92      0.90      0.91      7484



In [99]:
print(accuracy_score(pred_test_rf1, ytest), roc_auc_score(pred_test_rf1, ytest))

0.8981827899518974 0.74513195248665


In [100]:
rf2 = RandomForestClassifier()
rf2.fit(x_train_bw, y_train_bw)

pred_train_rf2 = rf2.predict(x_train_bw)
pred_test_rf2  = rf2.predict(xtest_bow)

In [101]:
print("Train Report: {}".format(classification_report(pred_train_rf2, y_train_tf)))
print("Test accuracy: {}".format(classification_report(pred_test_rf2, ytest)))

Train Report:               precision    recall  f1-score   support

           0       1.00      1.00      1.00     19941
           1       1.00      1.00      1.00     19921

    accuracy                           1.00     39862
   macro avg       1.00      1.00      1.00     39862
weighted avg       1.00      1.00      1.00     39862

Test accuracy:               precision    recall  f1-score   support

           0       0.54      0.54      0.54       829
           1       0.94      0.94      0.94      6655

    accuracy                           0.90      7484
   macro avg       0.74      0.74      0.74      7484
weighted avg       0.90      0.90      0.90      7484



In [102]:
print(accuracy_score(pred_test_rf2, ytest), roc_auc_score(pred_test_rf2, ytest))

0.8972474612506681 0.7410540520700127


In [103]:
rf2.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

### Model Selection
1. XGBoost with bag of words feature extraction performs best among all 4 models
2. The model's performance can be improved using feature selection/dimensionality reduction on the data and hyperparameter optimization of the model

#### Saving XGBoost Model and Bag-of-words model

In [104]:
import pickle

In [108]:
# pkl_filename = "tfidf_model.pkl"
# with open(pkl_filename, 'wb') as file:
#     pickle.dump(tfidf_model, file)
    
# pkl_filename = "bow_model.pkl"
# with open(pkl_filename, 'wb') as file:
#     pickle.dump(bow_model, file)

In [106]:
# pkl_filename = "xgb_tf_model.pkl"
# with open(pkl_filename, 'wb') as file:
#     pickle.dump(xgb1, file)

# pkl_filename = "xgb_bow_model.pkl"
# with open(pkl_filename, 'wb') as file:
#     pickle.dump(xgb2, file)

## Prediction Pipeline

In [ ]:
def process_reviews(reviews_list):
    pass

In [109]:
tf_model_file = "tfidf_model.pkl"
with open(tf_model_file, 'rb') as file:
    tf_model = pickle.load(file)
    
xgtf_model_file = "xgb_tf_model.pkl"
with open(xgtf_model_file, 'rb') as file:
    xgtf_model = pickle.load(file)

In [142]:
some = tf_model.transform(xtrain[1200:1201].to_list())
xgtf_model.predict_proba(some)

array([[0.2597574, 0.7402426]], dtype=float32)

In [143]:
ytrain[1200:1201]

15377    1
Name: user_sentiment, dtype: int64

## Hyper-parameter Optimization:
- XGBoost (Best Performing Model)
- Prevent Overfitting (Using Random Search with Cross Validation & PCA)

In [82]:
# n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
# max_features = ['auto', 'sqrt']
# max_depth = [int(x) for x in np.linspace(10, 100, num=10)]
# max_depth.append(None)
# min_samples_split = [2, 5, 10]
# min_samples_leaf = [1, 2, 4]
# bootstrap = [True, False]

# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

# print(random_grid)

In [83]:
# rf = RandomForestClassifier(random_state=42)

# rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
#                           n_iter=30, scoring='accuracy', 
#                           cv=3, verbose=2, random_state=42, n_jobs=-1,
#                           return_train_score=True)

# # Fit the random search model
# rf_random.fit(x_train, y_train)

In [84]:
# rf_random.best_params_